# JustWatch와 IMDb 간단한 데이터 전처리

In [210]:
# import
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
from tqdm import tqdm_notebook

from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import requests
import urllib.parse

In [211]:
# 엑셀 파일을 읽어옵니다.
df = pd.read_excel('../data/imdb_final.xlsx', index_col=0)
df.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,...,season_11,season_12,season_13,season_14,season_15,season_16,season_17,director,writer,actor
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Hwang Dong-hyuk'],['Hwang Dong-hyuk'],"['Lee Jung-jae', 'Park Hae-soo', 'Yasushi Iwak..."
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Kaat Beels', 'Wouter Bouvijn']","['Bert Van Dael', 'Nele Meirhaeghe', 'Roel Mon...","['Luc De Ruelle', 'Maaike Neuville', 'Tom Verm..."
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['David Chuang', 'Kuan-Chung Chen']","['Joyce Liu', 'Jui-Liang Hsu', 'Shih-Keng Chie...","['Wei-Ning Hsu', 'Hsiao-chuan Chang', 'Shih-Si..."
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Alexis Ostrander', 'Carol Banker', 'Ciarán F...","['Beth Schwartz', 'Bo Yeon Kim', 'Carly Woodwo...","['Nonso Anozie', 'Christian Convery', 'Stefani..."
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Jang Young-woo', 'Lee Eung-bok', 'Park So-hy...","['Hong So-ri', 'Hwang Young-Chan', 'Kim Hyung-...","['Song Kang', 'Lee Jin-wook', 'Lee Si-young', ..."


In [212]:
# 애니메이션, TV쇼 개별 삭제
df.drop(124, axis=0, inplace=True)
df.drop(151, axis=0, inplace=True)
df.drop(696, axis=0, inplace=True)
df.drop(826, axis=0, inplace=True)
df.drop(865, axis=0, inplace=True)
df.drop(873, axis=0, inplace=True)
df.drop(887, axis=0, inplace=True)
df.drop(913, axis=0, inplace=True)
df.drop(947, axis=0, inplace=True)
df.drop(1032, axis=0, inplace=True)
df.drop(1058, axis=0, inplace=True)

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1079 entries, 0 to 1089
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               1079 non-null   object 
 1   original_title      845 non-null    object 
 2   year                1079 non-null   int64  
 3   season_episode      1079 non-null   int64  
 4   runtime             1079 non-null   object 
 5   genre               1079 non-null   object 
 6   age_rating          798 non-null    object 
 7   Production country  1078 non-null   object 
 8   IMDb_title          1079 non-null   object 
 9   IMDb_URL            1079 non-null   object 
 10  total_rate          1077 non-null   float64
 11  total_count         1077 non-null   object 
 12  watchlist           841 non-null    object 
 13  popularity          503 non-null    object 
 14  user_review         847 non-null    object 
 15  critic_review       809 non-null    float64
 16  age_miss   

In [214]:
# '평점 없음'인 행들을 선택
rows_to_drop = df[df['season_1'] == '평점 없음'].index

# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)

# 시즌1 '평점 없음' 행 삭제 갯수
print(len(rows_to_drop))

153


In [215]:
# '2024년 공개작 포함'인 행들을 선택
rows_to_drop = df[df['season_1'] == '2024년 공개작 포함'].index
# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)
# 시즌1 '2024년 공개작 포함' 행 삭제 갯수
len(rows_to_drop)

10

In [216]:
# '에피소드 정보 없음'인 행들을 선택
rows_to_drop = df[df['season_1'] == '에피소드 정보 없음'].index
# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)
# 시즌1 '에피소드 정보 없음' 행 삭제 갯수
print(len(rows_to_drop))

7


In [217]:
# 'season_1' 컬럼의 값이 None인 행들을 선택
rows_to_drop = df[df['season_1'].isna()].index

# 삭제된 행의 수 출력
print(len(rows_to_drop))

# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)

0


In [218]:
df[795:805]

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,...,season_11,season_12,season_13,season_14,season_15,season_16,season_17,director,writer,actor
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
881,네 아이는 네 아이가 아니다,你的孩子不是你的孩子,2018,1,1시간 8분,"SF, 가족, 드라마",15,대만,Ni de hai zi bu shi ni de hai zi (2018),https://www.imdb.com/title/tt8507402/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Wei-ling Chen'],"['Chien-Tai Ma', 'Kang-Chen Hsia', 'Kung-Yi Fe...","['Tzu-Chuan Liu', 'Yu-Xuan Wang', 'Hannah Ring..."
883,섀도우,Shadow,2019,1,45min,"액션, 범죄, 드라마, 스릴러",NaN,남아프리카공화국,Shadow (TV Mini Series 2019),https://www.imdb.com/title/tt9817236/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Fred Wolmarans', 'Gareth Crocker']",['Gareth Crocker'],"['Pallance Dladla', 'Khathu Ramabulana', 'Aman..."
884,크리미널: 프랑스,Criminal: France,2019,1,26min,"드라마, 범죄, 스릴러",15,프랑스,Criminal: France (TV Mini Series 2019),https://www.imdb.com/title/tt10986052/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Frédéric Mermoud'],"['Antonin Martin-Hilbert', 'Frédéric Mermoud',...","['Margot Bancilhon', 'Laurent Lucas', 'Stéphan..."
885,섄티 타운,Shanty Town,2023,1,40min,"범죄, 드라마, 액션, 스릴러",NaN,나이지리아,Shanty Town (TV Series 2023,https://www.imdb.com/title/tt22099034/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Dimeji Ajibola'],"['Ayomikun Paseda', 'Donald Tombia', 'Shirleya...","['Richard Mofe-Damijo', 'Nse Ikpe-Etim', 'Chid..."
889,피의 시인,Bard of Blood,2019,2,45min,"액션, 드라마, 스릴러",NaN,인도,Bard of Blood (TV Series 2019,https://www.imdb.com/title/tt8069036/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Ribhu Dasgupta'],"['Bilal Siddiqi', 'Mayank Tewari', 'Vidit Trip...","['Reapstar', 'Gurii', 'Abhi Brar', 'Emraan Has..."
890,스트레인지 스토리,పిట్టా కథలు,2021,1,37min,"드라마, 로맨스",15,인도,Pitta Kathalu (TV Series 2021),https://www.imdb.com/title/tt13878800/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['B.V. Nandini Reddy', 'Nag Ashwin', 'Sankalp ...","['Lakshmi Bhupala', 'Nag Ashwin', 'Nanda Kisho...","['Shruti Haasan', 'Jagapathi Babu', 'Satyadev ..."
892,영혼 신부,彼岸之嫁,2020,1,51min,"스릴러, 드라마, 가족, 판타지",19,"말레이시아, 대만",The Ghost Bride (TV Mini Series 2020),https://www.imdb.com/title/tt9690302/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Shio Chuan Quek', 'Yuhang Ho']","['Anwari Ashraf', 'Dan Hamamura', 'June Tan', ...","['Peijia Huang', 'Kang Ren Wu', 'Ludi Lin', 'K..."
894,멈출 수 없어,Desenfrenadas,2020,1,34min,드라마,19,멕시코,Unstoppable (TV Series 2020),https://www.imdb.com/title/tt11120674/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Diego Martínez Ulanosky', 'Elisa Miller', 'J...","['Ashauri López', 'Diego Martínez Ulanosky', '...","['Tessa Ia', 'Bárbara López', 'Lucía Uribe', '..."
896,새비지 리듬,Ritmo salvaje,2022,1,38min,"드라마, 음악",NaN,콜롬비아,Savage Rhythm (TV Series 2022),https://www.imdb.com/title/tt14494806/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Andres Beltran', 'Rafael Martínez Moreno', '...","['Diego Vivanco', 'Juliana Barrera', 'Valeria ...","['Paulina Dávila', 'Brooke Newton', 'Angela Ca..."


In [219]:
# 컬럼명 rename
df.rename(columns={'Production country': 'production_country','season_episode':'total_season_num','IMDb_title':'imdb_title','IMDb_URL':'imdb_url'}, inplace=True)

In [220]:
# production_country 결측치 채우고 영어 이름(china) 한국이름으로 바꾸기
df['production_country'] = df['production_country'].fillna('China')

In [221]:
# 번역할 나라 이름 딕셔너리
countries_dict = {'China': '중국', 'Arjantin': '아르헨티나', '필리핀 제도': '필리핀'}

# 각 행의 값을 번역하는 함수
def translate_country(country):
    countries = country.split(', ')
    translated_countries = (countries_dict.get(c, c) for c in countries)
    return ', '.join(translated_countries)

In [222]:
df['production_country'].unique()

array(['대한민국', '벨기에', '대만', '미국', '덴마크', '독일', '일본', '태국', '캐나다', '그리스',
       '영국', '스페인', '미국, 영국', '아이슬란드, 덴마크, 핀란드, 스웨덴, 노르웨이, 독일, 프랑스, 영국',
       '미국, 프랑스', 'China', '호주', '스웨덴', '이탈리아', '케냐', '인도', '호주, 미국',
       '스위스', '필리핀 제도', '영국, 미국', '남아프리카공화국', '폴란드', '브라질', '미국, 캐나다',
       '콜롬비아', '터키', '룩셈부르크', '핀란드', '이스라엘', '프랑스', '멕시코', '벨기에, 네덜란드',
       '덴마크, 노르웨이', '노르웨이', '러시아', '미국, 일본', '영국, 프랑스', '요르단', '프랑스, 미국',
       '캐나다, 영국', '인도네시아', '영국, 인도', 'Arjantin', '아이슬란드', '오스트리아', '포르투갈',
       '프랑스, 독일, 스페인, 영국', '아일랜드, 영국', '캐나다, 프랑스, 독일', '이스라엘, 노르웨이',
       '대한민국, 미국', '미국, 대한민국', '네덜란드', '멕시코, 스페인', '칠레', '뉴질랜드, 호주',
       '사우디아라비아', '나이지리아', '이집트', '일본, 미국', '미국, 멕시코', '쿠웨이트',
       '나이지리아, 미국', '말레이시아, 대만', '사우디아라비아, 인도', '벨기에, 룩셈부르크',
       '사우디아라비아, 튀니지, 이집트, 모로코, 레바논', '독일, 오스트리아', '체코 공화국, 독일',
       '영국, 스페인', '스페인, 브라질', '레바논'], dtype=object)

In [223]:
df.head()

,title,original_title,year,total_season_num,runtime,genre,age_rating,production_country,imdb_title,imdb_url,...,season_11,season_12,season_13,season_14,season_15,season_16,season_17,director,writer,actor
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Hwang Dong-hyuk'],['Hwang Dong-hyuk'],"['Lee Jung-jae', 'Park Hae-soo', 'Yasushi Iwak..."
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Kaat Beels', 'Wouter Bouvijn']","['Bert Van Dael', 'Nele Meirhaeghe', 'Roel Mon...","['Luc De Ruelle', 'Maaike Neuville', 'Tom Verm..."
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['David Chuang', 'Kuan-Chung Chen']","['Joyce Liu', 'Jui-Liang Hsu', 'Shih-Keng Chie...","['Wei-Ning Hsu', 'Hsiao-chuan Chang', 'Shih-Si..."
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Alexis Ostrander', 'Carol Banker', 'Ciarán F...","['Beth Schwartz', 'Bo Yeon Kim', 'Carly Woodwo...","['Nonso Anozie', 'Christian Convery', 'Stefani..."
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Jang Young-woo', 'Lee Eung-bok', 'Park So-hy...","['Hong So-ri', 'Hwang Young-Chan', 'Kim Hyung-...","['Song Kang', 'Lee Jin-wook', 'Lee Si-young', ..."


### production_country 리스트형

In [224]:
'''
# 번역할 나라 이름 딕셔너리
countries_dict = {'China': '중국', 'Arjantin': '아르헨티나', '필리핀 제도': '필리핀'}

# 각 행의 값을 번역하는 함수
def translate_country(country):
    if isinstance(country, list):
        countries = country
    else:
        countries = country.split(', ')
    translated_countries = [countries_dict.get(c, c) for c in countries]
    return translated_countries

# 'production_country' 컬럼의 값을 번역
df['production_country'] = df['production_country'].apply(translate_country)

# 고유한 나라 리스트를 얻기 위한 집합
unique_countries = set()

# 각 행의 리스트에서 나라를 추가
for country_list in df['production_country']:
    unique_countries.update(country_list)

# 결과를 리스트로 변환
unique_countries = list(unique_countries)

# 결과 출력
print(unique_countries)
'''

"\n# 번역할 나라 이름 딕셔너리\ncountries_dict = {'China': '중국', 'Arjantin': '아르헨티나', '필리핀 제도': '필리핀'}\n\n# 각 행의 값을 번역하는 함수\ndef translate_country(country):\n    if isinstance(country, list):\n        countries = country\n    else:\n        countries = country.split(', ')\n    translated_countries = [countries_dict.get(c, c) for c in countries]\n    return translated_countries\n\n# 'production_country' 컬럼의 값을 번역\ndf['production_country'] = df['production_country'].apply(translate_country)\n\n# 고유한 나라 리스트를 얻기 위한 집합\nunique_countries = set()\n\n# 각 행의 리스트에서 나라를 추가\nfor country_list in df['production_country']:\n    unique_countries.update(country_list)\n\n# 결과를 리스트로 변환\nunique_countries = list(unique_countries)\n\n# 결과 출력\nprint(unique_countries)\n"

In [225]:
# df.head()

In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 909 entries, 0 to 1089
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               909 non-null    object 
 1   original_title      726 non-null    object 
 2   year                909 non-null    int64  
 3   total_season_num    909 non-null    int64  
 4   runtime             909 non-null    object 
 5   genre               909 non-null    object 
 6   age_rating          653 non-null    object 
 7   production_country  909 non-null    object 
 8   imdb_title          909 non-null    object 
 9   imdb_url            909 non-null    object 
 10  total_rate          909 non-null    float64
 11  total_count         909 non-null    object 
 12  watchlist           818 non-null    object 
 13  popularity          500 non-null    object 
 14  user_review         811 non-null    object 
 15  critic_review       781 non-null    float64
 16  age_miss    

In [227]:
# 'min'을 '분'으로 바꾸는 함수
def replaceMin(string):
    return string.replace('min', '분')

# 시간을 분으로 변환하는 함수
def getTime(runtime):
    # '시간'과 '분'을 기준으로 분리
    hours, minutes = 0, 0
    if '시간' in runtime:
        hours = int(runtime.split('시간')[0].strip())
    if '분' in runtime:
        if '시간' in runtime:
            minutes = int(runtime.split('시간')[1].split('분')[0].strip())
        else:
            minutes = int(runtime.split('분')[0].strip())
    # 시간과 분을 합쳐서 분으로 계산
    total_minutes = hours * 60 + minutes
    return total_minutes

# 'min'을 '분'으로 변경
df['runtime'] = df['runtime'].apply(replaceMin)
# 'runtime' 컬럼의 값을 변환
df['runtime'] = df['runtime'].apply(getTime)

In [228]:
# 수식어에 대한 값을 변환하는 함수
def convertValue(value):
    try:
        # ',' 제거
        value = value.replace(',', '')
        # 숫자와 수식어 분리
        if 'K' in value:
            multiplier = 1000
            value = float(value.replace('K', ''))
        elif 'M' in value:
            multiplier = 1000000
            value = float(value.replace('M', ''))
        elif '만' in value:
            multiplier = 10000
            value = float(value.replace('만', ''))
        elif '천' in value:
            multiplier = 1000
            value = float(value.replace('천', ''))
        else:
            multiplier = 1
            value = float(value)
        # 값에 수식어를 곱하고 정수로 변환하여 반환
        return int(value * multiplier)
    except AttributeError:
        # 숫자가 아닌 값이 들어올 경우 그대로 반환
        return value

# 해당 컬럼들의 값을 변환
df['total_count'] = df['total_count'].apply(convertValue)
df['watchlist'] = df['watchlist'].apply(convertValue)
df['popularity'] = df['popularity'].apply(convertValue)
df['user_review'] = df['user_review'].apply(convertValue)
df['critic_review'] = df['critic_review'].apply(convertValue)

In [229]:
df.head()

,title,original_title,year,total_season_num,runtime,genre,age_rating,production_country,imdb_title,imdb_url,...,season_11,season_12,season_13,season_14,season_15,season_16,season_17,director,writer,actor
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,Ojingeo Geim,Squid Game,2021,2,55,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Hwang Dong-hyuk'],['Hwang Dong-hyuk'],"['Lee Jung-jae', 'Park Hae-soo', 'Yasushi Iwak..."
1,12인의 심판자,The Twelve,2019,2,54,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Kaat Beels', 'Wouter Bouvijn']","['Bert Van Dael', 'Nele Meirhaeghe', 'Roel Mon...","['Luc De Ruelle', 'Maaike Neuville', 'Tom Verm..."
2,희생자 게임,誰是被害者,2020,2,60,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['David Chuang', 'Kuan-Chung Chen']","['Joyce Liu', 'Jui-Liang Hsu', 'Shih-Keng Chie...","['Wei-Ning Hsu', 'Hsiao-chuan Chang', 'Shih-Si..."
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Alexis Ostrander', 'Carol Banker', 'Ciarán F...","['Beth Schwartz', 'Bo Yeon Kim', 'Carly Woodwo...","['Nonso Anozie', 'Christian Convery', 'Stefani..."
4,Sweet Home,NaN,2020,3,60,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Jang Young-woo', 'Lee Eung-bok', 'Park So-hy...","['Hong So-ri', 'Hwang Young-Chan', 'Kim Hyung-...","['Song Kang', 'Lee Jin-wook', 'Lee Si-young', ..."


In [230]:
# 리스트가 아닌 문자열로 저장됨
print(len(df['production'][1]))
print(df['production'][1][0])

64
[


In [231]:
# 문자열을 리스트로 변환하는 함수
def parseStringToList(string):
    if pd.isna(string):  # NaN 값인 경우 빈 리스트 반환
        return []
    # 문자열에서 각 요소를 구분하는 쉼표로 분할
    elements = string.split(', ')
    # 각 요소에서 양쪽의 작은따옴표 제거
    elements = [element.strip("['").strip("']") for element in elements]
    return elements

# 'production','distributor' 컬럼의 값을 변환
df['production'] = df['production'].apply(parseStringToList)
df['distributor'] = df['distributor'].apply(parseStringToList)

In [232]:
# 리스트가 아닌 문자열로 저장됨
print(len(df['production'][1]))
print(df['production'][1][0])

2
Vlaamse Radio en Televisie (VRT)


In [233]:
# 저장
df.to_csv('../data/imdb_final_2.csv', sep="/", encoding="utf-8")

df.to_excel('../data/imdb_final_2.xlsx')

In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 909 entries, 0 to 1089
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               909 non-null    object 
 1   original_title      726 non-null    object 
 2   year                909 non-null    int64  
 3   total_season_num    909 non-null    int64  
 4   runtime             909 non-null    int64  
 5   genre               909 non-null    object 
 6   age_rating          653 non-null    object 
 7   production_country  909 non-null    object 
 8   imdb_title          909 non-null    object 
 9   imdb_url            909 non-null    object 
 10  total_rate          909 non-null    float64
 11  total_count         909 non-null    int64  
 12  watchlist           818 non-null    float64
 13  popularity          500 non-null    float64
 14  user_review         811 non-null    float64
 15  critic_review       781 non-null    float64
 16  age_miss    

In [234]:
# 인덱스 변경
# df.reset_index(drop=True,inplace=True)# 새로운 인덱스 부여

In [ ]:
# english_title, age_rating_us, total_season_num(IMDb기준)